# Computational Methods in Simulation (CMIS) Week 2
This notebook is intended to help students make their hand-in in this week of CMIS.


Hand-ins must be submitted as a maximum 3 page pdf file in Absalon and must be made using the ACM TOG overleaf latex template

https://www.overleaf.com/latex/templates/association-for-computing-machinery-acm-large-2-column-format-template/qwcgpbmkkvpq

Notebooks must be uploaded as well so teachers have code solutions accessible if needed.

This week is based on Slides 11 and 12

* Advection Problem: From Page 61 in slides 11

    * Define some φ field on a regular 2D grid (Hint: The function peaks might be useful).

    * Use the analytical velocity field u(x,y) = (y,−x)T

    * Explain in detail how to use semi-Lagrangian time integration to implement a scheme for solving ∂φ/∂t = − (u · ∇) φ on a regular mesh domain.

    * Ideally φ should just rotate as a function of time. Implement your derived scheme and examine if this is true. (Hint: You have to carefully define an experiment and identify what causes φ to change).
    
* Mean Curvatue Flow: From page 17 in Slides 12

    * Discus what boundary conditions to apply to φ for a mean curvature flow. Assume that φ is a signed distance field. (HINT: Try and sketch a signed distance map in 1D and draw a vertical line at some imaginary boundary).

    * Implement a mean curvature flow simulation with the purpose of making it as robust as possible.

    * Assume that input φ is a signed distance map. Examine if this property holds through out a simulation.

    * Analyse which of the “Numerical Remedies” that are most important to achieve a robust simulation.

## Detailed Expected Learning Objectives in this Week

* Finite Difference Methods (FDM) Part 2 (Lecture slides number 11)
    * Apply Semi Lagrangian Implicit Time integration to advection type problems.
* Finite Difference Methods (FDM) Part 3 (Lecture slides number 12)
    * Apply FDM techniques on a larger more complex case-study.

## Common Data Structures
This week is studying two different types of PDE problems. One is advection by a rotational velocity
field where we investigate semi-Lagrangian time-integration and the other is a geometry
problem of changing a shape of a polygon by using mean curvature flow.

* Advection with Semi-Lagrangian Time Integration
* Mean Curvature Flow

Both these applications make use of a regular 2D grid to store a value field that in the assignment text was labelled by $\phi$. Hence, we will start by creating a simple grid data structure that will help us make our code a little more easy later on.

First we will make sure all pyhton packages we need are loaded and any helper functions are set up.

In [1]:
%matplotlib widget

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.mplot3d import Axes3D

def vec2(x,y):
    """
    Helper functions for quickly creating 2D vectors without having to write too much text.
    """
    return np.array([x,y], dtype=np.float64)

Next we are ready to define our grid data structure. It is quite simple as it uses three 2D arrays. One for storing x-coordinates of all nodes in the grid, one for the y-coordinates and one for all the values stored at the node. The grid must support 3 different kind of use cases:

* The constructor makes sure to allocate these 2D arrays and setup any other member variables that we may need later on. Such as the cell spacing, number of nodes along each axis etc.

* The interpolate method is responsible for querying field values from non-node grid locations. That is this method reconstructs the field value at a given location p by using the enclosing nodal values and linear basis functions.

* The last funcgtionality is the ability to easily map a function onto the grid. That is we want to pass the grid a function that is evaluated for all node coordinates and the function values are then stored in the grid value array.


In [3]:
class SimpleGrid2D:
    
    def __init__(self, min_coord, max_coord, I, J):
        if np.any(min_coord > max_coord):
            raise ValueError()
        if I < 2:
            raise ValueError()
        if J < 2:
            raise ValueError()

        self.min_coord = min_coord
        self.max_coord = max_coord
        
        self.I = I  # Number of nodes along x axis
        self.J = J  # Number of nodes along y axis
        
        res = np.array([I-1, J-1], dtype=np.float64)
        dims = (self.max_coord - self.min_coord)
        self.spacing = np.divide( dims, res)
        
        self.dx = self.spacing[0]
        self.dy = self.spacing[1]

        self.x = np.linspace(min_coord[0], max_coord[0], I)
        self.y = np.linspace(min_coord[1], max_coord[1], J)
    
        X, Y = np.meshgrid(self.x, self.y)
        self.X = X.T
        self.Y = Y.T
        self.values = np.zeros((I, J),dtype=np.float64)
        
    def interpolate(self, p):
        idx = np.floor((p - self.min_coord) / self.spacing)
        # if index is outside the cells then project onto closest cell
        i = np.clip(int(idx[0]), 0, self.I-2)
        j = np.clip(int(idx[1]), 0, self.J-2)
                
        x00 = self.values[i,j]
        x01 = self.values[i+1,j]
        x10 = self.values[i,j+1]
        x11 = self.values[i+1,j+1]
        
        s = (p[0]  - (i * self.spacing[0] + self.min_coord[0]) ) / self.spacing[0]
        t = (p[1]  - (j * self.spacing[1] + self.min_coord[1])) / self.spacing[1]        

        y0 = (x01 - x00) * s + x00
        y1 = (x11 - x10) * s + x10
        return (y1 - y0) * t + y0
    
    def eval_func(self, func):
        for i in range(self.I):
            for j in range(self.J):
                x = self.X[i,j]
                y = self.Y[i,j]
                self.values[i,j] = func(vec2(x,y))

As our grid data structure uses numpy arrays we must be careful if we want to create a copy of a grid value. Hence, we have a special functions for creating proper copies. This is to make sure that no two variable names share the same storage. 

In [4]:
def make_copy(grid):
    cpy = SimpleGrid2D(grid.min_coord, grid.max_coord, grid.I, grid.J)
    cpy.values = np.copy(grid.values)
    return cpy

Once we start populating our grid data structure with values from different functions then it is helpful to be able to easy visualize the values stored in a grid. For this purpose we have a function that simply show the grid values as an image.

In [5]:
def show_grid(grid):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_title('Grid Values')
    plt.imshow(grid.values)
    ax.set_aspect('equal')
    plt.colorbar(orientation='vertical')
    plt.show()

We only need a few functions that can help us put some values into a grid where we know exactly what the results should be. We have created a peaky looking shape function that is nice for advection problems simply because the peaks will suffer greatly from any numerical dissipation in ones scheme. The other function is a trivial linear function. It is nice for testing if our interpolation method works as bilinear interpolation should be exact for a linear function sampled on a grid, even if the grid is very coarse.

In [6]:
def peaks(p):
    x = p[0]
    y = p[1]    
    z = 3*((1-x)**2)*np.exp(-(x**2) - (y+1)**2) - 10*(x/5 - x**3 - y**5)*np.exp(-x**2 - y**2) - 1/3*np.exp(-(x+1)**2 - y**2)
    return z


def ramp(p):
    return p[0]

Now we make test if our copy function work as intended

In [7]:
min_coord = vec2(0, 0)
max_coord = vec2(1, 1)
A = SimpleGrid2D(min_coord, max_coord, 4, 4)
A.eval_func( peaks )
print("A is now peak = ", A.values)

plt.imshow(A.values)
plt.show()

B = make_copy(A)
A.eval_func( ramp )
print("A is now ramp = ", A.values)
print("B is still peak = ", B.values)

A is now peak =  [[ 0.98101184  0.43413361  0.95225665  3.68862957]
 [ 0.11744864 -0.05306361  0.62362037  3.19551845]
 [ 1.10278675  0.9761935   1.21134321  2.73945317]
 [ 2.93693032  2.64162775  2.19372215  2.43378912]]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

A is now ramp =  [[0.         0.         0.         0.        ]
 [0.33333333 0.33333333 0.33333333 0.33333333]
 [0.66666667 0.66666667 0.66666667 0.66666667]
 [1.         1.         1.         1.        ]]
B is still peak =  [[ 0.98101184  0.43413361  0.95225665  3.68862957]
 [ 0.11744864 -0.05306361  0.62362037  3.19551845]
 [ 1.10278675  0.9761935   1.21134321  2.73945317]
 [ 2.93693032  2.64162775  2.19372215  2.43378912]]


Next we verify the interpolation function

In [8]:
p = vec2(0.1,0.1)
value = A.interpolate( p)
print("point = ", p," expected = ", ramp(p), " actual = ", value)

p = vec2(0.1,0.9)
value = A.interpolate( p)
print("point = ", p," expected = ", ramp(p), " actual = ", value)

p = vec2(0.5,0.5)
value = A.interpolate( p)
print("point = ", p," expected = ", ramp(p), " actual = ", value)

p = vec2(0.9,0.1)
value = A.interpolate( p)
print("point = ", p," expected = ", ramp(p), " actual = ", value)

point =  [0.1 0.1]  expected =  0.1  actual =  0.1
point =  [0.1 0.9]  expected =  0.1  actual =  0.1
point =  [0.5 0.5]  expected =  0.5  actual =  0.5
point =  [0.9 0.1]  expected =  0.9  actual =  0.9


# Advection with Semi-Lagrangian Implicit Time Integration
We know have our common data structures in place and we can start solving the first part of this hand-in. Below is our implementation of the Semi-Lagrangian Implicit time-integration. The code tries to test the relative error of a conservation merit value for different grid spacings and timestep sizes. The purpose is to investigate how the error behaves when we decreate spacing and time-step sizes.

A critical task is to define a meaningfull measure of conservation. We have used the total positive volume as a measure of conservation. That is

$A(t) \equiv \int_S |\phi(t)| dS \approx \sum_{i,j} |\phi_{i,j}(t)| \Delta x \Delta y$

Now we may define the relative error as 

$e(T) \equiv \frac{|A(T)-A(0)|}{|A(0)|}$

Here $T$ is the end-time of a simulation and we pick this such that the advection should make one full revolution. In a perfect ideal world there would be no dissipation and no numerical rounding etc. Hence, $\phi(T) = \phi(0)$. In the real world this is not so and our error measure tries to measure how good the real world is at not loosing too much information in $\phi(T)$.

Besides the error measurements we also store the end-results of the computations for later visualizations

In [30]:
def plo(grid):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.plot_surface(grid.X, grid.Y, grid.values, cmap="magma", color='0.75', rstride=1, cstride=1)
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_zlabel("u")
    plt.show()

min_coord = vec2(-5, -5)
max_coord = vec2(5, 5)

grid = SimpleGrid2D(min_coord, max_coord, 200, 200)
grid.eval_func( peaks )

plo(grid)

dt = 1
for i in range(grid.I):
    for j in range(grid.J):
        
        xt = vec2(grid.X[i,j], grid.Y[i,j])
        u  = vec2(grid.Y[i,j], -grid.X[i,j])
        
        xt_prev = xt - dt * u
        
        phi = cpy.interpolate(xt_prev)

        grid.values[i, j] = phi

plo(grid)


C:\Users\DKMEQ\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

C:\Users\DKMEQ\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [68]:
I_values = np.logspace(5, 8, 5, base=2.0).astype(int)
dt_values = np.logspace(-5, -8, 5, base=2.0)

print(I_values)
print(dt_values)

[ 32  53  90 152 256]
[0.03125    0.01858136 0.01104854 0.0065695  0.00390625]


In [ ]:
# I_values = [40, 80, 100, 120, 140] # start with a single value for a fast turn around, then later expand this
# dt_values = [0.03, 0.02, 0.01, 0.005] # same here

N = len(I_values)
K = len(dt_values)

error = np.zeros((N,K), dtype=np.float64)

for n in range(N):
    I = I_values[n]
    J = I
        
    min_coord = vec2(-5, -5)
    max_coord = vec2(5, 5)
        
    for k in range(K):
        grid = SimpleGrid2D(min_coord, max_coord, I, J)
        grid.eval_func( peaks )
        dT = dt_values[k]
        T = 2.0*np.pi
        
        print("Running on grid (I,J) = (", I, J, "), total time = ", T, " and timestep dt = ", dT)
    
        merit_0 = np.sum(np.abs( grid.values.flatten() )) * grid.dx * grid.dy
        
        while T>0:
            dt = np.min([dT, T])
            cpy = make_copy(grid)
            for i in range(grid.I):
                for j in range(grid.J):
                    xt = vec2(grid.X[i,j], grid.Y[i,j])
                    u  = vec2(grid.Y[i,j], -grid.X[i,j])

                    xt_prev = xt - dt * u

                    phi = cpy.interpolate(xt_prev)

                    grid.values[i, j] = phi

            T = T - dt
        
        merit_T = np.sum(np.abs( grid.values.flatten() ))*grid.dx*grid.dy
        error[n,k] = np.abs(merit_T - merit_0) / np.abs(merit_0)
        
        print(" error = ", error[n,k])

print("Done comparing numerical parameters")

Running on grid (I,J) = ( 32 32 ), total time =  6.283185307179586  and timestep dt =  0.03125
 error =  0.5444887252082855
Running on grid (I,J) = ( 32 32 ), total time =  6.283185307179586  and timestep dt =  0.018581361171917516
 error =  0.5235397937583393
Running on grid (I,J) = ( 32 32 ), total time =  6.283185307179586  and timestep dt =  0.011048543456039806
 error =  0.5122374119721775
Running on grid (I,J) = ( 32 32 ), total time =  6.283185307179586  and timestep dt =  0.0065695032441696445
 error =  0.5057318419718292
Running on grid (I,J) = ( 32 32 ), total time =  6.283185307179586  and timestep dt =  0.00390625
 error =  0.5019941648652095
Running on grid (I,J) = ( 53 53 ), total time =  6.283185307179586  and timestep dt =  0.03125
 error =  0.46740974691170634
Running on grid (I,J) = ( 53 53 ), total time =  6.283185307179586  and timestep dt =  0.018581361171917516
 error =  0.43554196528479794
Running on grid (I,J) = ( 53 53 ), total time =  6.283185307179586  and ti

Now we are ready to visualize the error. We expect the error to improve when spacing and time-step descreases.

In [46]:
DT, DI = np.meshgrid(dt_values, I_values)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(DI, DT, error, cmap="magma", color='0.75', rstride=1, cstride=1)
ax.set_xlabel("#N")
ax.set_ylabel("dt")
ax.set_zlabel("error")
plt.show()


C:\Users\DKMEQ\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  This is separate from the ipykernel package so we can avoid doing imports until


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Signed Distance Fields
Before we can start on the mean curvature assignment we are in need of a function that can compute a signed distance field.

The next function takes a query point q as input and a closed polygon, poly, and computes the signed distance between q and poly. If the value is positive the q is outside the polygon if the value is negative then q is inside the polygon. The polygon is represented its N-corners and these are stored in an (N+1)-by-2 numpy array. First column is x-coordinates and second column is y-coordinates. The first and last row should be the same corner inorder to close the polygon.

This function is convenient when evaluating a signed distance field on a grid.

In [9]:
def compute_signed_distance(poly, q):
    
    def is_left(p0, p1, p2):
        return (p1[0] - p0[0]) * (p2[1] - p0[1]) - (p2[0] - p0[0]) * (p1[1] - p0[1])

    N = poly.shape[0]
    d = np.inf

    # Compute distance to closest surface point
    for i in range(N-1):
        a = poly[i, :]    # Point on the surface of polygon p
        b = poly[i+1, :]  # Adjacent point on the surface of polygon p
        
        ba = b - a     # Vector from a to b
        bal = np.linalg.norm(ba) # Length of ba vector
        if bal == 0:
            continue
        ban = ba / bal # Normal of the vector from a to b
        qa = q - a     # The vector from the query point to a
        alpha = np.dot(qa, ban) / bal
        if alpha < 0:
            da = np.linalg.norm(q-a)
            if da < d:
                d = da
        elif alpha > 1:
            db = np.linalg.norm(q-b)
            if db < d:
                d = db
        else:
            banp = np.array([-ban[1], ban[0]])
            dp = abs(np.dot(qa, banp))
            if dp < d:
                d = dp
    # Compute winding number to set the sign
    wn = 0
    for i in range(N-1):
        a = poly[i, :]    # Point on the surface of polygon p
        b = poly[i+1, :]  # Adjacent point on the surface of polygon p
        if a[1] <= q[1]:
            if b[1] > q[1]:
                if is_left(a, b, q) > 0:
                    wn = wn + 1
        else:
            if b[1] <= q[1]:
                if is_left(a, b, q) < 0:
                    wn = wn -1
    if wn < -0.5:
        d = -d
    elif wn > 0.5:
        d = -d
    return d

# Mean Curvature Flow
The central part of the mean curvature flow assignment is to be able to evaluate the value of the mean curvature by using higher order finite difference approximations and dealing properly with boundary conditions and any numerical remedies that would be needed for stability.

We have created a single function that takes a grid as input assuming this grid is holding a signed distance field and then the function computes the mean curvature field of the signed distance field.

Your task is to write the function based on the lecture 12 slides.

In [39]:
# MEAN_CURVATURE_FLOW - Mean Curvature Flow
# INPUT:
#      grid - initial signed distance map
# RESULT:
#      kappa - the mean curvature field of the signed distance field
def compute_mean_curvature(grid):
    cpy = make_copy(grid)

    for i in range(1, grid.I - 1):
        for j in range(1, grid.J - 1):
            
            Dx = (grid.values[i + 1, j] - grid.values[i - 1, j]) / (2 * grid.dx)
            Dy = (grid.values[i, j + 1] - grid.values[i, j - 1]) / (2 * grid.dy)
            
            Dxx = (grid.values[i + 1, j] - 2 * grid.values[i, j] + grid.values[i - 1, j]) / (grid.dx ** 2)
            Dyy = (grid.values[i, j + 1] - 2 * grid.values[i, j] + grid.values[i, j - 1]) / (grid.dy ** 2)
            Dxy = (grid.values[i + 1, j + 1] - grid.values[i + 1, j - 1] - grid.values[i - 1, j + 1] + grid.values[i - 1, j - 1]) / (4 * grid.dx * grid.dy)
             
            enum = (Dx ** 2) * Dyy + (Dy ** 2) * Dxx - 2 * Dxy * Dx * Dy 
            
            g = np.sqrt(Dx ** 2 + Dy ** 2) 
            
            if not g > 0.5:
                g = 1
            
            cpy.values[i, j] = enum / ((g ** 3) + 0.0000001)
        
    for i in range(0, grid.I):
        cpy.values[i, 0] = cpy.values[i, 1]
        cpy.values[i, grid.J - 1] = cpy.values[i, grid.J - 2]

    for j in range(0, grid.J):
        cpy.values[0, j] = cpy.values[1, j]
        cpy.values[grid.I - 1, j] = cpy.values[grid.I - 2, j]

    kappa = cpy.values

    return kappa

To test our curvature field we must first create a grid instance that holds a signed distance field

In [40]:
poly = np.zeros((5,2),dtype=np.float64)
poly[0,:] = vec2(-1,-1)
poly[1,:] = vec2( 1,-1)
poly[2,:] = vec2( 1, 1)
poly[3,:] = vec2(-1, 1)
poly[4,:] = vec2(-1,-1)
        
min_coord = vec2(-2, -2)
max_coord = vec2(2, 2)
    

grid = SimpleGrid2D(min_coord, max_coord, 64, 64)
grid.eval_func( lambda p: compute_signed_distance(poly,p) )

show_grid(grid)

C:\Users\DKMEQ\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In order to plot, let's save the curvature field we start with in another variable. We'll call it original. 

In [41]:
original = make_copy(grid)

All pieces are now in place to setup the mean curvature flow main loop. This loop evolves the distance field using the mean curvature field as the speed function. Once we are done we plot contours of the original and the resulting distance fields to see the effect on the shape. 

In [42]:
T = 0.5
dt = 0.001
while T>0:
    kappa = compute_mean_curvature(grid)
    grid.values = grid.values + dt*kappa
    T = T - dt
    
show_grid(grid)

print(grid.values)

fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_title('Mean Curvature Field from Last Iteration')
plt.imshow(kappa)
ax.set_aspect('equal')
ax.set_xlabel('i')
ax.set_ylabel('j')

fig = plt.figure()
ax = fig.add_subplot(111)
plt.contour(original.X, original.Y, original.values, [0.0], colors='r')
plt.contour(grid.X, grid.Y, grid.values, [0.0], colors='b')
ax.set_title('Countour comparison')
ax.set_aspect('equal')
ax.set_xlabel('x')
ax.set_ylabel('y')
plt.show()

C:\Users\DKMEQ\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[[1.82658708 1.78242721 1.73961958 ... 1.73961958 1.78242721 1.82658708]
 [1.78242721 1.73679574 1.69247103 ... 1.69247103 1.73679574 1.78242721]
 [1.73961958 1.69247103 1.64659444 ... 1.64659444 1.69247103 1.73961958]
 ...
 [1.73961958 1.69247103 1.64659444 ... 1.64659444 1.69247103 1.73961958]
 [1.78242721 1.73679574 1.69247103 ... 1.69247103 1.73679574 1.78242721]
 [1.82658708 1.78242721 1.73961958 ... 1.73961958 1.78242721 1.82658708]]


C:\Users\DKMEQ\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  if sys.path[0] == '':


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

C:\Users\DKMEQ\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

That's it folks!